# Load New Dataset - Test Prediction Accuracy

In [78]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

import lesco as lsc

## Load Data

In [79]:
root = r'C:\Users\XPC\Desktop\single-frame'

classes_ = []
files_ = []

for path, subdirs, files in os.walk(root):
    for name in files:
        file_path = path + "\\" + name
        img_class = name.split('_')[1].split('.')[0]
        files_.append(file_path)
        classes_.append(img_class)
        
print(len(files_), "video images detected")

336 video images detected


## Augment Dataset

In [80]:
X = []
y = []
count = 0

for url, class_ in zip(files_, classes_):
    
    try:

        # code from notebook 1
        img = cv.imread(url)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        v1 = lsc.process_image(img, False, False, 0)
        v2 = lsc.process_image(img, True, False, 0)
        v3 = lsc.process_image(img, False, True, 5)
        v4 = lsc.process_image(img, False, True, -5)
        v5 = lsc.process_image(img, True, True, 5)
        v6 = lsc.process_image(img, True, True, -5)

        if (v1 is None):
            print("No hands at", url)

        if(v1 is not None):
            X.append(v1)
            y.append(class_)

        if(v2 is not None):
            X.append(v2)
            y.append(class_)

        if(v3 is not None):
            X.append(v3)
            y.append(class_)

        if(v4 is not None):
            X.append(v4)
            y.append(class_)

        if(v5 is not None):
            X.append(v5)
            y.append(class_)

        if(v6 is not None):
            X.append(v6)
            y.append(class_)

        count = count + 1

        #if (count % 10 == 0):
        #    clear_output(wait=True)
    except Exception as e:
        print("error",str(e))
    
print("Total samples created", len(X), len(y))

No hands at C:\Users\XPC\Desktop\single-frame\022965aa-2fa0-4bbf-a2b3-bcfd38aca605_desmayar.png
No hands at C:\Users\XPC\Desktop\single-frame\064dc251-0707-482d-9f15-1e283fdad11c_calle.png
No hands at C:\Users\XPC\Desktop\single-frame\0f49fb95-4f54-4402-8bdd-39b43625e4b8_fuego.png
No hands at C:\Users\XPC\Desktop\single-frame\1c5306b0-8b4f-48b5-aaec-2ca2385996c9_cocina.png
No hands at C:\Users\XPC\Desktop\single-frame\271cfe1d-ed51-4a34-9def-b8e4b81adc37_basura.png
No hands at C:\Users\XPC\Desktop\single-frame\27bd62e0-bfb1-4e5f-9393-e5b174b58dbc_casa.png
No hands at C:\Users\XPC\Desktop\single-frame\285e0319-1a31-4000-afa6-910e71c201c7_casa.png
No hands at C:\Users\XPC\Desktop\single-frame\2f798c09-5e73-49ea-9872-98e6a22fcd13_yo.png
No hands at C:\Users\XPC\Desktop\single-frame\31a31d50-bd8b-465e-abed-edc0a1beeee5_fuego.png
No hands at C:\Users\XPC\Desktop\single-frame\35046a2f-2b54-4462-97ca-6e98be8ae3d1_tener.png
No hands at C:\Users\XPC\Desktop\single-frame\35bc935d-4b4f-45e2-9403-

## Prediction

In [81]:
# test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=43)

print("Training", len(X_train))
print("Training", len(X_test))

Training 1308
Training 328


In [82]:
def get_manhattan(train_data, test_item):
    similarity = []
    for data in train_data:
        s = 1 - np.abs(np.array(data) - np.array(test_item)).sum()
        similarity.append(s)
        
    df = pd.DataFrame({
        "y_train" : y_train,
        "similarity" : similarity
    })
    
    df = df.sort_values(by=['similarity'], ascending=False)
    
    return df.iloc[0]["y_train"]  # returns most similar class

In [83]:
total = len(X_test)
count = 0

for test, label in zip(X_test, y_test):
    y_pred = get_manhattan(X_train, test)
    
    if (y_pred == label):
        count = count + 1

print("Manhattan Normal Fn Test Set Accuracy:", round(count/total,2))

Manhattan Normal Fn Test Set Accuracy: 0.8


In [84]:
scores = []
for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    total = len(X_test)
    count = 0
    for test, label in zip(X_test, y_test):
        y_pred = get_manhattan(X_train, test)
        if (y_pred == label):
            count = count + 1
    scores.append(round(count/total,2))
print("Scores:", scores)
print("Manhattan Cross Validation", np.mean(scores))

Scores: [0.81, 0.8, 0.79, 0.76, 0.78]
Manhattan Cross Validation 0.788


# Video Sentence Prediction

Every frame of the video is classified. Then if there are trends of common words, they will be used as the idenfied token words

## Load Lesco Sentences

In [85]:
root = r'C:\Users\XPC\Desktop\LESCO-oraciones'

files_ = []

for path, subdirs, files in os.walk(root):
    for name in files:
        class_name = os.path.basename(path)
        file_path = path + '\\' + name
        files_.append(file_path)
        
print(len(files_),"emergency videos detected")

21 emergency videos detected


In [86]:
# they are burning trash
video_url = files_[0]
print(video_url)

cap = cv.VideoCapture(video_url)
while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        break
    frame = cv.resize(frame, (480,300), interpolation = cv.INTER_AREA)            
    cv.imshow('frame', frame)

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

C:\Users\XPC\Desktop\LESCO-oraciones\bomberos_Estan quemando basura.mp4


## Video Frame Classification

In [181]:
def get_video_words(files_, index):
    i = 0
    i_succ = 0
    identified_classes = []

    video_url = files_[index]
    print(video_url)

    cap = cv.VideoCapture(video_url)
    while cap.isOpened():
        ret, frame = cap.read()
        # if frame is read correctly ret is True
        if not ret:
            break

        frame = cv.resize(frame, (480,300), interpolation = cv.INTER_AREA)            

        try:
            # get hands metadata
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            lesco_array = lsc.process_image(frame, False, False, 0)

            if (type(lesco_array) != type(None)):
                y_pred = get_manhattan(X_train, lesco_array)
                identified_classes.append(y_pred)
                i_succ += 1

        except Exception as e:
            print(str(e))

        i += 1

    cap.release()

    print("Total Frames Processed", i)
    print("Total Classifications", i_succ)
    return identified_classes

In [199]:
def predict_sentence(identified_classes, threshold):
    ln = len(identified_classes)
    i = 0

    while i < ln:
        word = identified_classes[i]
        word_count = 1

        z = i
        for j in range(z, ln):
            if (word == identified_classes[j]):
                word_count += 1
            else:
                i = i + word_count
                break

        if (word_count > threshold):
            print(word, word_count, i, i + word_count, ln) 
            
        if (i + word_count > ln):
            break
        

In [200]:
# Estan quemando basura

identified_classes = get_video_words(files_, 0)  
print(" ")
predict_sentence(identified_classes, 14) 

C:\Users\XPC\Desktop\LESCO-oraciones\bomberos_Estan quemando basura.mp4
Total Frames Processed 323
Total Classifications 265
 
ellos 26 46 72 265
fosforo 15 93 108 265
basura 28 258 286 265


## Predict Second Sentence

In [205]:
# Están cortando árboles sin permiso

identified_classes = get_video_words(files_, 1)  
print(" ")
predict_sentence(identified_classes, 4)  

C:\Users\XPC\Desktop\LESCO-oraciones\bomberos_Están cortando árboles sin permiso.mp4
Total Frames Processed 245
Total Classifications 100
 
carro 5 61 66 100
nopermiso 29 72 101 100
